# Simulated TOD Data Generation

This notebook walks you through generating a synthetic dataset of focal-plane images and segmentation masks from time-ordered detector data (TOD).

You will:

1. **Import** all necessary libraries and custom helper modules.
2. **Configure** simulation parameters (scan geometry, noise levels, event rates, etc.).
3. **Set up** your output directories and manifest file.
4. **Define** helper functions for indexing and saving PNGs.
5. **Run** the main loop to produce batches of images with point-source and cosmic-ray events, saving both the grayscale frames and colorized masks, and logging metadata to `manifest.csv`.

**Prerequisites:**

* You’re already on the remote server.
* Activate the `glitchenv` conda environment:

  ```bash
  conda activate glitchenv
  ```
* Open this notebook in VS Code’s Jupyter interface.

Adjust any paths or dataset names in Cell 3 before running. Let’s get started!

---

# Generation Script

## Cell 1: Imports


In [1]:
# Load necessary libraries and modules
import numpy as np                             # numerical operations
import csv                                     # CSV file I/O
from itertools import product                   # Cartesian product for loops
from pathlib import Path                        # filesystem paths
from PIL import Image                           # image saving

# Custom simulation and mapping modules
from context import src
import src.simulatedDataGenerationHelper as simGeneration
import src.simulationDataMappingHelper as simMapping

## Cell 2: Simulation parameters

In [ ]:
# — Units & geometry
deg = np.deg2rad(1)                              # one degree in radians
arcmin = deg / 60                                # one arcminute in radians

# — Focal-plane geometry
FP_RADIUS = 0.8 * deg                            # radius of focal plane in radians
FP_NROWS = 30                                    # number of detector rows in focal plane

# — Scan timing & pointing
SIM_T0 = 1672531200.0                            # start time (Unix epoch)
SIM_AZ_START = 45 * deg                          # starting azimuth angle
SIM_EL = 60 * deg                                # elevation angle
SIM_AZ_THROW = 10 * deg                          # azimuth sweep half-width
SIM_V_AZ = 2 * deg                               # azimuth scan speed (rad/s)
SIM_SRATE = 200.0                                # sample rate (Hz)
SIM_DURATION = 120                               # duration of scan (s)

# — TOD-model parameters
WHITE_NOISE_LEVEL = 0.2                          # white noise level per detector
POINT_SRC_BEAM_SIGMA = 4 * arcmin                # point-source beam width
POINT_SRC_FLUX_LIMS = (1, 3)                     # min/max flux for point sources
CR_AMP_SCALE = 5                                 # cosmic-ray amplitude scale
CR_RADIUS_SCALE = 2 * arcmin                     # cosmic-ray radius scale

# — Data generation settings
seeds = [42]                          # seed for reproducibility
n_srcs_list = [5]                                # number of point sources to simulate
cr_rate_list = [0.01]                            # cosmic-ray event rate per sample

GRID_RESOLUTION = 32                             # resolution of grid for mapping
CHUNK_SEC = 1                                    # seconds per frame chunk (→ SIM_SRATE * CHUNK_SEC frames)


## Cell 3: Output setup & manifest

In [ ]:
# — Output naming & directories
BASE_OUTPUT_PATH = Path("/home/aziz/suds/suds2025_documentation/data/simulated_data")  # base folder for all datasets
DATASET_NAME = "output_dataset_120"                       # name for this generation run


OUTPUT_DIR = BASE_OUTPUT_PATH / DATASET_NAME                      # full path for this run's outputs
IMG_DIR = OUTPUT_DIR / "images"                                  # directory for saved images
MASK_DIR = OUTPUT_DIR / "masks"                                  # directory for saved masks

# Create directories if they don't exist
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
IMG_DIR.mkdir(parents=True, exist_ok=True)
MASK_DIR.mkdir(parents=True, exist_ok=True)

# Prepare manifest CSV to record metadata for each batch
manifest_path = OUTPUT_DIR / "manifest.csv"
fieldnames = ["batch_id", "seed", "n_srcs", "cr_rate", "start_idx", "end_idx", "tod_file"]
with open(manifest_path, "w", newline="") as mf:
    writer = csv.DictWriter(mf, fieldnames=fieldnames)
    writer.writeheader()


## Cell 4: Helpers & build geometry

In [4]:
# Function to find next image index in a folder
def get_next_index(folder: Path) -> int:
    idxs = []
    for f in folder.glob("img_*.png"):
        parts = f.stem.split("_")
        if len(parts) == 2 and parts[1].isdigit():
            idxs.append(int(parts[1]))
    return max(idxs, default=-1) + 1


# Function to save batches of images and masks
def save_batches(batches, out_img: Path, out_mask: Path):
    idx = get_next_index(out_img)

    # Color palette mapping label IDs to RGB colors
    palette = {
        0: (30, 100, 180),    # background (no event)
        1: (220, 20, 60),     # cosmic-ray
        2: (255, 105, 180),   # point-source
        3: (0, 206, 209),     # both events
    }

    for b in batches:
        for imgs, masks in zip(b["images"], b["masks"]):
            # Normalize and convert image to 8-bit grayscale
            arr = np.asarray(imgs, float)
            norm = (arr - arr.min()) / (np.ptp(arr) or 1)
            img = Image.fromarray((norm * 255).astype(np.uint8), mode="L")

            # Build RGB mask image using color palette
            m_arr = np.asarray(masks, np.uint8)
            rgb = np.zeros((*m_arr.shape, 3), np.uint8)
            for cid, col in palette.items():
                rgb[m_arr == cid] = col
            mask = Image.fromarray(rgb, mode="RGB")

            # Save files with zero-padded index
            name = f"{idx:05d}.png"
            img.save(out_img / f"img_{name}")
            mask.save(out_mask / f"mask_{name}")
            idx += 1

# Build focal-plane geometry once
fplane = simGeneration.FocalPlane.from_radius(
    radius=FP_RADIUS,
    nrows=FP_NROWS,
)

## Cell 5: Generation & saving loop

In [5]:
batch_id = 0  # initialize batch counter
for seed, n_srcs, cr_rate in product(seeds, n_srcs_list, cr_rate_list):
    # 1) Generate time-ordered data (TOD)
    np.random.seed(seed)
    scan = simGeneration.generate_ces_scan(
        t0=SIM_T0,
        az=SIM_AZ_START,
        el=SIM_EL,
        az_throw=SIM_AZ_THROW,
        v_az=SIM_V_AZ,
        srate=SIM_SRATE,
        duration=SIM_DURATION,
    )
    tod_models = [
        simGeneration.WhiteNoise(
            nlevs=np.ones(fplane.n_dets) * WHITE_NOISE_LEVEL
        ),
        simGeneration.PointSourceSimulator(
            n_srcs=n_srcs,
            beam_sigma=POINT_SRC_BEAM_SIGMA,
            flux_limits=POINT_SRC_FLUX_LIMS,
            save_mask=True,
        ),
        simGeneration.CosmicRaySimulator(
            n_per_sample=cr_rate,
            amp_scale=CR_AMP_SCALE,
            radius_scale=CR_RADIUS_SCALE,
            save_mask=True,
        ),
    ]
    tod, store = simGeneration.build_tod(
        fplane=fplane,
        scan=scan,
        tod_models=tod_models,
        n_dummy=50,
    )

    # 2) Map TOD to video frames and build labels
    flex = simMapping.FocalPlaneFlex(
        fplane=fplane,
        grid_resolution=GRID_RESOLUTION,
    )
    video = flex.tod_to_video(tod.data)                   # detector signals
    chunk = int(CHUNK_SEC * SIM_SRATE)                    # frames per chunk
    imgs = np.array([
        np.max(video[i:i+chunk], axis=0)
        for i in range(0, len(video), chunk)
    ])
    cr_v = flex.tod_to_video(store["tod_cr_mask"])
    pt_v = flex.tod_to_video(store["tod_ptsrc_mask"])
    lbl_cr = np.array([
        np.sum(cr_v[i:i+chunk], axis=0)
        for i in range(0, len(cr_v), chunk)
    ])
    lbl_pt = np.array([
        np.sum(pt_v[i:i+chunk], axis=0)
        for i in range(0, len(pt_v), chunk)
    ])
    labels = (
        (lbl_cr > 0).astype(int)
        + (lbl_pt > 0).astype(int) * 2
    )

    # 3) Save images, masks and record in manifest
    start_idx = get_next_index(IMG_DIR)
    save_batches(
        [{"images": imgs.tolist(), "masks": labels.tolist()}],
        IMG_DIR,
        MASK_DIR,
    )
    end_idx = get_next_index(IMG_DIR) - 1
    with open(manifest_path, "a", newline="") as mf:
        writer = csv.DictWriter(mf, fieldnames=fieldnames)
        writer.writerow({
            "batch_id": batch_id,
            "seed": seed,
            "n_srcs": n_srcs,
            "cr_rate": cr_rate,
            "start_idx": start_idx,
            "end_idx": end_idx,
            "tod_file": f"tod_{batch_id:02d}.npy",
        })

    print(f"[Batch {batch_id}] imgs {start_idx:05d}–{end_idx:05d}")
    batch_id += 1


RA bounds: 52.23 to 57.84 deg
Dec bounds: -5.09 to 0.01 deg
Sky map shape: (612, 673), wcs: car:{cdelt:[-0.008333,0.008333],crval:[55.03,0],crpix:[337.13,611.00]}
No sky models provided, initializing TOD with zeros.
Sky map shape: (612, 673), tod shape: (648, 24000)
Simulating 5 point sources within footprint
	Adding point source at RA: 52.85 deg, Dec: -0.94 deg
	Adding point source at RA: 56.90 deg, Dec: -4.40 deg
	Adding point source at RA: 56.78 deg, Dec: -3.66 deg
	Adding point source at RA: 55.79 deg, Dec: -3.69 deg
	Adding point source at RA: 55.92 deg, Dec: -3.05 deg
Generating 244 cosmic ray hits.
Triangulation created with 1236 triangles
Weight matrix created with 1689 non-zero elements
Valid grid points: 568 / 1024
[Batch 0] imgs 00000–00119
